# Статистические эксперименты и проверка гипотез

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

## Доверительные интервалы

Как мы видели, для разных выборок оценка параметров (и описательные статистики) различаются, даже если распеделение одинаково. Таким образом, оценка - тоже случайная величина, со своим распределением и статистикой.

Возьмем какой-нибудь интервал вокруг нашей оценки $\bar\theta$:

$$
\mathcal{I} = \left[\bar\theta - \epsilon, \bar\theta + \epsilon \right]
$$

Какова вероятность, что настоящий параметр лежит внутри? 

$$
\mathbb{P}[\theta \in \mathcal{I}] = \mathbb{P}[\bar\theta - \epsilon \leqslant \theta \leqslant \bar\theta + \epsilon]
$$

На практике мы обычно определяем $\epsilon$, для которого $\mathbb{P}[\theta \in \mathcal{I}] \geqslant 1 - \alpha$, где $\alpha$ называется **уровнем значимости**. Иными словами, при $\alpha = 0.05$ получится 95% **доверительный интервал**.

Важно не забывать, что интервал строится вокруг нашей оценки, а не истинного параметра! Он может оказаться в любом месте интервала, или, с вероятностью в пределах $\alpha$, вне его.

### Доверительный интервал для среднего

Как следствие из ЦПТ, выборочное среднее стремится распределиться как

$$
\bar X \sim \mathcal{N}(\mu, \sigma^2/n)
$$

Или, что то же самое, 

$$
\sqrt{n}\frac{\bar X - \mu}{\sigma} \sim \mathcal{N}(0,1)
$$

Такую нормализованную величину обычно обозначают буквой $Z$.

Как мы помним, начение $z$, при котором площадь под кривой равна $\alpha$, это ее PPF от $\alpha$. Поскольку стандартная гауссиана симметрична относительно нуля, нам легко найти и интервал $[-z, z]$, где площадь под кривой соответствует $1-\alpha$: это $[-PPF((1 - \alpha)/2), +PPF((1 - \alpha)/2)]$.

Или, говоря проще, 95% ближайших к центру значений $\sim \mathcal{N}(0,1)$ лежат в этом промежутке.

Вернемся к формуле выше: если говорить упрощенно, ошибка $(\bar X - \mu)$ распределена нормально, как $Z\frac{\sigma}{\sqrt{n}}$, другими словами, наш искомый интервал будет:

$$
\mathcal{I} = \left[\bar x - PPF_Z(1 - \alpha/2)\frac{\sigma}{\sqrt{n}},\text{   } \bar x + PPF_Z(1 -\alpha/2)\frac{\sigma}{\sqrt{n}}\right]
$$

Для краткости *критические значения* вроде $PPF_Z(1 - \alpha)$ обычно записывают как $z_\alpha$.

In [ ]:
X = np.random.exponential(2, 100)
Z = st.norm(0, 1)
z = Z.ppf(1 - 0.05/2)
print(f'95% доверительный интервал среднего: {X.mean() - z * 4 / 10} .. {X.mean() + z * 4 / 10}')

Величина $\frac{\sigma}{\sqrt{n}}$ называется **стандартной ошибкой**. Как видно, чем больше выборка, тем уже интервал.

In [ ]:
st.norm.interval(0.95, loc=X.mean(), scale=4/10)

### Распределение Стьюдента

Если перейти к выборочной дисперсии, то

$$
\sqrt{n}\frac{\bar X - \mu}{s}
$$

Уже не будет распределено нормально. Тем не менее, эта величина также имеет характерное распределение - распределение Стьюдента.

In [ ]:
X1 = st.norm(0,1)
X2 = st.t(1)
X3 = st.t(5)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
plt.suptitle('PDF и CDF распределения Стьюдента')
ax[0].set(xlabel = 'Значения X', ylabel = 'Плотность вероятности')
ax[1].set(xlabel = 'Верхний предел X', ylabel = 'Вероятность')

points = np.linspace(-5, 5, 100)
pdf1 = np.array([X1.pdf(x) for x in points])
ax[0].plot(points, pdf1);
pdf2 = np.array([X2.pdf(x) for x in points])
ax[0].plot(points, pdf2);
pdf3 = np.array([X3.pdf(x) for x in points])
ax[0].plot(points, pdf3);
ax[0].legend(['Нормальное', '$t_1$', '$t_5$'])

cdf1 = np.array([X1.cdf(x) for x in points])
ax[1].plot(points, cdf1);
cdf2 = np.array([X2.cdf(x) for x in points])
ax[1].plot(points, cdf2);
cdf3 = np.array([X3.cdf(x) for x in points])
ax[1].plot(points, cdf3);

Как видно, форма очень похожа на нормальную, но "хвосты" распредения тяжелее. У него один параметр - число степеней свободы, с ростом которого оно приближается к нормальному.

При достаточно большом $n$

$$
\sqrt{n}\frac{\bar X - \mu}{s} \sim t(n-1)
$$

#### Самостоятельная работа

Вычислите 95% доверительный интервал среднего для уже объявленной выборки X. Используйте вычисление выше как образец.
Вам понадобится `scipy.stats.t(n-1)`.

In [ ]:
# Ваш код ниже:


Проверьте результат с помощью scipy:

In [ ]:
st.t.interval(0.95, 99, loc=X.mean(), scale=st.sem(X))

### Bootstrap

Как построить доверительный интервал, если распределение оценки неизвестно?

In [ ]:
X = np.random.exponential(1/2, 10000)

In [ ]:
estimates = []
for i in range(1000):
    sample = np.random.choice(X, len(X))
    estimates.append(np.median(sample))

z005 = st.norm(0,1).ppf(1-0.05/2)
print(np.mean(estimates) - z005 * np.std(estimates, ddof=1), np.mean(estimates) + z005 * np.std(estimates, ddof=1))

In [ ]:
np.median(X)

In [ ]:
st.expon(scale=1/2).median()

#### Код для воспроизводимого параллельного бутстрепа

In [ ]:
from joblib import Parallel, delayed

def bootstrap(func, X, estimator, iterations, random_state=177013, n_jobs=12):
    state = np.random.RandomState(random_state)
    estimates = Parallel(n_jobs)(delayed(func)(X, estimator, state) for i in range(iterations))
    result = st.norm(estimator(estimates), np.std(estimates, ddof=1)).interval(0.95)
    return result


def boot_classic(X, estimator, state):
    sample = state.choice(X, len(X))
    return estimator(sample)

def boot_bayes(X, estimator, state):
    w = np.random.dirichlet(np.ones(len(X)) * 4, 1)[0]
    sample = np.random.choice(X, len(X), p=w)
    return estimator(sample)

In [ ]:
X = np.random.exponential(1/2, 10000)

In [ ]:
bootstrap(boot_classic, X, np.median, 1000)

In [ ]:
bootstrap(boot_bayes, X, np.median, 1000)

In [ ]:
b = st.bootstrap((X,), np.median, confidence_level=0.95, n_resamples=1000, random_state=177013)

In [ ]:
b.confidence_interval

## Проверка статистических гипотез

Как формулируются гипотезы?

Гипотеза - то, что можно проверить с помощью наблюдений.

Прежде всего, нужна **нулевая гипотеза ($H_0$)** - гипотеза об отсутствии эффекта. Математически она описывается как равенство. *Отвергая* нулевую гипотезу, мы подтвердим эффект.

Например: "Средние двух распределений равны", "параметр распределения равен 0.5".

**Альтернативная гипотеза ($H_1$)** - гипотеза о наличии эффекта, о неравенстве. Она может быть как односторонней ("параметр больше 0.5"), так и двусторонней ("средние выборок не равны").

После формулировки гипотез следует определиться с **уровнем значимости $\alpha$**. Он соответствует наибольшей вероятности **ошибки первого рода** (ложноположительного срабатывания, обнаружения эффекта там, где его нет), другими словами, того, что мы отвергнем верную нулевую гипотезу.

Третьим этапом определяется **статистический критерий** - какая-то статистика выборки и соответствующие ей *критические области*. При подадании значения статистики в эти области мы отвергаем нулевую гипотезу.

### Z-тест для среднего

Положим, есть некое распределение, для которого мы знаем дисперсию $\sigma^2$, но среднее $\mu$ можем только оценить.

Определим гипотезы так:

$$
H_0: \mu = \mu_0 \\
H_1: \mu \neq \mu_0
$$

Тогда чем ближе $\mu_0$ к истине, тем меньше $\bar X - \mu_0$, а мы помним, что

$$
\sqrt{n}\frac{\bar X - \mu_0}{\sigma}
$$

стремится распределиться нормально.

То есть критические значения соответствуют $PPF_Z(1 - \alpha)$, что для двустороннего теста выглядит так:

$$
\left|\sqrt{n}\frac{\bar X - \mu_0}{\sigma}\right| \geqslant z_{\alpha/2}
$$

В этих областях мы отвергаем нулевую гипотезу.

Как видим, принцип очень похож на тот, что мы использовали для построения доверительного интервала. Действительно, отвергнуть нулевую гипотезу - это то же самое, что сказать "$\mu_0$ не попадает в соответствующий доверительный интервал для $\mu$". 

Аналогично для одностороннего теста:

$$
H_1: \mu > \mu_0 : \text{отвергаем }H_0\text{, если статистика}\geqslant z_{\alpha}\\
H_1: \mu < \mu_0 : \text{отвергаем }H_0\text{, если статистика}\leqslant -z_{\alpha}
$$

### T-тест для среднего

Все то же самое, если мы не знаем дисперсию: используем распределение Стьюдента.

Критерий:
$$
\sqrt{n}\frac{\bar X - \mu_0}{S}
$$

Критические значения^ $\pm t(n-1)_{\alpha/2}\text{ или}\pm t(n-1)_{\alpha}$ для односторонних тестов.

### P-value

На практике проще проверять не вхождение в интервал, а *вероятность получить данное или более экстремальное значение критерия, если нулевая гипотеза верна*. Если оно меньше порога $\alpha$, то нулевая гипотеза отвергается.

In [ ]:
X = np.random.poisson(2, 100)

alpha = 0.05

In [ ]:
criterion_value, p_value = st.ttest_1samp(X, 1)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

**Для одностороннего теста достаточно разделить P-value на 2, а больше/меньше видно по выборочным средним.**

### A/B тестирование

#### Z-критерий для двух выборок:

$$
\large{\frac{\bar X - \bar Y}{\sqrt{\frac{\sigma^2_x}{n_x}+\frac{\sigma^2_y}{n_y}}}}
$$

#### Т-критерий для двух выборок:

$$
\large{\frac{\bar X - \bar Y}{\sqrt{\frac{s^2}{n_x}+\frac{s^2}{n_y}}}}
$$

Где $s$ считается по выборочной дисперсии:

$$
s = \sqrt{\frac{(n_x-1)s^2_x + (n_y-1)s^2_y}{n_x+n_y-2}}
$$

А число степеней свободы будет $n_x+n_y-2$.

#### Т-тест Уэлча
Если у распределений разные дисперсии, то формула упрощается:

$$
\large{\frac{\bar X - \bar Y}{\sqrt{\frac{s^2_x}{n_x}+\frac{s^2_y}{n_y}}}}
$$

А число степеней свободы в простейшем случае можно брать как $min{(n_x, n_y)} -1$ (но есть и более сложные методы).

Этот тест рекомендуется использовать по умолчанию.

In [ ]:
X = np.random.poisson(2, 100)
Y = np.random.poisson(2.5, 100)

In [ ]:
criterion_value, p_value = st.ttest_ind(X, Y, equal_var=False)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

### Тест для зависимых выборок

In [ ]:
before = np.random.poisson(2, 100)
after = np.random.poisson(2.5, 100)

In [ ]:
criterion_value, p_value = st.ttest_rel(before, after)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

### Допущения и свойства параметрических тестов

- Z-тест и Т-тест опираются на ЦПТ. Выборка должна быть достаточно большой (или маленькой, но нормально распределенной).
- Как следствие, выбросы также сильно влияют на надежность тестов.

**Ошибка первого рода**, как мы уже говорили, - вероятность отвергнуть верную нулевую гипотезу. **Ошибка второго рода**, соответственно, - когда нам не удалось отвергнуть неверную нулевую гипотезу (то есть мы не обнаружили эффект там, где он есть). Верхний предел на них обозначается $\alpha$ и $\beta$.

**Статистической мощностью теста** называется вероятность обнаружить эффект, когда он присутствует ($1-\beta$).

**Минимальный размер эффекта** для А/B теста - это минимальное различие между двумя статистиками, которое тест можзет обнаружить. От него зависит минимальный нужный размер выборки.

Для одностороннего Т-теста это:

$$
n \geqslant \left(\frac{(z_a+z_b)^2}{MDE}(s^2_x+s^2_y)\right)
$$

Для двустороннего:

$$
n \geqslant \left(\frac{(z_{a/2}+z_b)^2}{MDE}(s^2_x+s^2_y)\right)
$$

### Непараметрические тесты

Непараметрические тесты стоит применять, если:

- выборка мала;
- нет возможности удалить выбросы и/или медианные значения важнее средних.

Непараметрические тесты сортируют и ранжируют значения после какой-то обработки. Эти ранги и используются для подсчета критерия. Они используют лишь значения выборок и их количество. Для них не важны параметры распределения. 

#### U-тест Манна-Уитни

Его можно рассматривать как непараметрический аналог Т-теста для независимых выборок. Он проверяет гипотезу о равенстве распределений.

In [ ]:
criterion_value, p_value = st.mannwhitneyu(X, Y)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

#### W-тест Уилкоксона

Его можно рассматривать как непараметрический аналог Т-теста для зависимых выборок. Он проверяет гипотезу о симметричности распределения разности относительно нуля.

In [ ]:
criterion_value, p_value = st.wilcoxon(before, after)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

#### Тест Муда на равенство медиан

In [ ]:
X = np.random.poisson(1, 100)
Y = np.random.poisson(1.4, 100)

In [ ]:
criterion_value, p_value, grand_median, _ = st.median_test(X, Y)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

#### Тесты на нормальность

In [ ]:
X = np.random.normal(5, 10, 100) + np.random.uniform(-1, 1, 100)

In [ ]:
plt.hist(X);

Тест Шапиро-Уилка (неплохо работает с малыми выборками):

In [ ]:
criterion_value, p_value, = st.shapiro(X)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

Тест Андерсона-Дарлинга позволяет протестировать разные формы распределений по критическим значениям (образает особое внимание на хвосты):

In [ ]:
criterion_value, critical_value, levels = st.anderson(X, dist='norm')

print(f'Результат: {criterion_value}')
print(f'Критические значения {critical_value}')
print(f'Уровни достоверности, % {levels}')

Тест Колмогорова-Смирнова тестирует на качество попадания в любое распределение по оценке CDF (чувствителен и к среднему, и к дисперсии):

In [ ]:
criterion_value, p_value = st.kstest(X, st.norm(5, 10).cdf)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

### Другие полезные тесты

#### Тесты на равенство дисперсий

In [ ]:
X = np.random.normal(5, 10, 100) + np.random.uniform(-10, 10, 100)
Y = np.random.normal(5, 10, 100)

In [ ]:
plt.hist(X);
plt.hist(Y);

In [ ]:
criterion_value, p_value = st.bartlett(X, Y)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

In [ ]:
criterion_value, p_value = st.levene(X, Y)

print(f'P-value {p_value}')
if p_value < alpha:
    print('Отвергаем нулевую гипотезу.')
else:
    print('Не удается опровергнуть нулевую гипотезу.')

# Домашнее задание

Вернемся к датасету клиентов банка.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Churn_Modelling.csv')

In [ ]:
loyal = df.query('Exited == 0')
exited = df.query('Exited == 1')

## Easy

Вычислите двусторонние 95% доверительные интервалы для среднего по столбцу `CreditScore` в выборках `loyal` и `exited`, сделанных выше.

In [ ]:
# Ваш код ниже:


## Normal

Завершите исследование: проведите тесты на равенство средних в выборках `loyal` и `exited` по следующим столбцам:
- `CreditScore`
- `Age`
- `Tenure`

Примите уровень достоверности в 5%.

In [ ]:
# Ваш код ниже:


Напишите краткий вывод:


# Hard

Вычислите одностороннний 95% доверительный интервал (в большую сторону) для минимума по столбцу `EstimatedSalary` методом bootstrap.

In [ ]:
# Ваш код ниже:
